# Transformers 를 이용해 Backbone 사용

## Transformers 설치
- `pip install transformers`

In [10]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


### Tokenizer, Model Loading
- Huggingface 모델 허브에서 제공하는 처리 모델을 다운받아 로딩한다.
- 다운로드된 모델은 `사용자 home 디렉토리\.cache\huggingface` 에 저장된다.
- 미리 학습된 언어 모델을 다운받아 사용할 때는 그 언어모델이 사용한 tokenizer를 같이 받아서 사용한다.

### [Auto Classes](https://huggingface.co/docs/transformers/model_doc/auto)
- Huggingface 에서 제공하는 다양한 모델들은 손쉽게 불러오고 사용할 수 있도록 설계된 유틸리티 클래스들을 말한다.
- 미리 학습된 특정 모델의 이름(모델 허브상에서 제공되는 이름)이나 저장된 local 경로를 제공하면 해당 모델에 맞는 적절한 클래스와 구성 요소를 자동으로 로드한다.
- 사용자는 모델을 사용하기 위한 정확한 클래스를 몰라도 쉽게 다양한 종류의 모델을 사용할 수있다.

#### 주요 Auto Class
- 기본 모델 Loading  

    1. **AutoModel**
       - 주어진 모델 이름에 맞는 사전 학습된 모델을 자동으로 로드한다.
       - 예: `AutoModel.from_pretrained("bert-base-uncased")`: BERT 모델을 로드한다.  

    2. **AutoTokenizer**
       - 해당 모델에 적합한 토크나이저를 자동으로 로드한다.
       - 예: `AutoTokenizer.from_pretrained("bert-base-uncased")`: BERT 모델에 맞는 토크나이저를 로드한다.  
       
    3. **AutoConfig**
       - 모델의 설정(config)을 자동으로 로드한다. 모델 설정에는 모델의 하이퍼파라미터와 모델 구조 정보가 포함된다. 이 설정을 이용해 모델을 생성할 수있다.  

       - 예: `AutoConfig.from_pretrained("bert-base-uncased")`  

- Task 처리 모델 Loading
    - Pretrained backbone 모델에 각 task 에 맞는 estimator layer를 추가한 모델을 생성해 제공한다.
    - 주요 모델들
        1. **AutoModelForSequenceClassification**
           - 시퀀스(Text) 분류 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")`  

        2. **AutoModelForQuestionAnswering**
           - 질문-응답 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")`   
           
        3. **AutoModelForTokenClassification**
           - 토큰 분류 작업(예: 개체명 인식)을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForTokenClassification.from_pretrained("bert-base-uncased")`

In [11]:
from transformers import AutoModel, AutoConfig, AutoTokenizer

In [7]:
model_id = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_id)
print(type(tokenizer))    # pretrained 된 토크나이저
model = AutoModel.from_pretrained(model_id)
print(type(model))   # pretrained 된 모델
config = AutoConfig.from_pretrained(model_id)
print(type(config))   # pretrained 된 모델 설정.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\Playdata\AppData\Local\miniconda3\envs\NLP\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
<class 'transformers.models.bert.modeling_bert.BertModel'>
<class 'transformers.models.bert.configuration_bert.BertConfig'>


In [8]:
from pprint import pprint

text = "I am a boy."
tokens = tokenizer(
    text,
    return_tensors="np",  # 결과의 타입 지정함.(pt: torch.tensor, tf: tensorflow.tensor, np: ndarray, default: list)
)
pprint(tokens)
print(tokens['attention_mask'])
# attention_mask: 각 토큰이 실제 토큰인지 (1), padding(0) 구분하는 값들로 구성.
# input_ids: 토큰 id
# token_type_ids : 입력으로 두개 문장을 받았을때 몇번째 문장의 토큰인지를 구분. (0, 1)

{'attention_mask': array([[1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': array([[ 101, 1045, 2572, 1037, 2879, 1012,  102]]),
 'token_type_ids': array([[0, 0, 0, 0, 0, 0, 0]])}
[[1 1 1 1 1 1 1]]


In [9]:
tokenizer.encode(text)  # 토큰 id 만 반환.

[101, 1045, 2572, 1037, 2879, 1012, 102]

In [ ]:
## 여러 문서(문장)들을 한번에 토큰화
### -> max_length, truncation 설정
### -> dafault 토큰 길이 : 개별 문장별 길이에 맞춰 토큰 생성.
text_list = ["I am a boy", "I am hungry.", "I am going to home."]
token_list = tokenizer(
    text_list,
    max_length=7,    #최대 토큰 수
    padding=True,    # padding 추가
    truncation=True  # max_lenghth 가 넘을 경우 짤라낸다.
)
pprint(token_list)

{'attention_mask': [[1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[101, 1045, 2572, 1037, 2879, 102],
               [101, 1045, 2572, 7501, 1012, 102],
               [101, 1045, 2572, 2183, 2000, 2188, 1012, 102]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0]]}


In [12]:
token_list = tokenizer(
    text_list,
    # max_length=7,    #최대 토큰 수
    padding=True,    # max_length 설정 없이 'padding=True' -> 제일 긴 토큰을 가진 문서에 맞춘다.
    # truncation=True  # max_lengh?th 가 넘을 경우 짤라낸다.
)
pprint(token_list)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0],
                    [1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[101, 1045, 2572, 1037, 2879, 102, 0, 0],
               [101, 1045, 2572, 7501, 1012, 102, 0, 0],
               [101, 1045, 2572, 2183, 2000, 2188, 1012, 102]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0]]}


In [18]:
## 토크나이저의 정보를 조회
print("총 어휘개수:", tokenizer.vocab_size, len(tokenizer))
print("token_id -> token 단어:", tokenizer.convert_ids_to_tokens(0))
print("token_id -> token 단어:", tokenizer.convert_ids_to_tokens([100, 3500, 2750]))
print("token_단어 -> token_id:", tokenizer.convert_tokens_to_ids("spring"))
print("token_단어 -> token_id:", tokenizer.convert_tokens_to_ids(["spring", "i", "am"]))
# tokenizer.get_vocab() # 모든 토큰들

총 어휘개수: 30522 30522
token_id -> token 단어: [PAD]
token_id -> token 단어: ['[UNK]', 'spring', 'despite']
token_단어 -> token_id: 3500
token_단어 -> token_id: [3500, 1045, 2572]


### tokenizer 에 토큰들을 추가
- tokenizer와 모델을 같이 받았고, tokenizer에 토큰들을 추가했을 경우 이것을 모델에 적용시켜야 한다.  

- 모델의 embedding vector의 크기를 재조정해야 하기 때문이다.

In [20]:
# 추가
print(len(tokenizer))
tokenizer.add_tokens(["미이이이아라", "티치터치티칭", "##커치커피컴", "뿌ㅇㄹㄹ리링", "adlskdhlk"])
# 사이즈
print(len(tokenizer))

30522
30527


In [ ]:
# 모델 업데이트 (단어를 추가했기 때문에 모델도 업데이트가 필요함.)
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(30527, 768, padding_idx=0)

## kcbert
- BERT 모델을 한글 텍스트로 학습 시킨 Pretrained model.
    - BERT는 Transformer의 Encoder 부분을 이용해 구현된 언어모델
    - https://arxiv.org/abs/1810.04805 
- https://huggingface.co/beomi/kcbert-base

### 토크나이저 모델 load

In [ ]:
from transformers import AutoModel, AutoTokenizer


model_id = "beomi/kcbert-base"     # base model => Feature Extractor
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)


In [ ]:
text_list = ["안녕하세요. 반갑습니다.", 
             "kcbert는 bert 모델을 한국어로 학습한 모델입니다.", 
             "토크나이저와 모델은 같은 ID의 것으로 받아야 합니다."]

In [24]:
token_list = tokenizer(
    text_list,
    max_length=10,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

In [26]:
print(type(token_list))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [28]:
pprint(token_list)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[    2, 19017,  8482,    17,  1483,  4981,  8046,    17,     3,     0],
        [    2,    76,  4773,  4545, 13146,  4401,  4008,    67, 13146,     3],
        [    2,  3160,  4147, 16991,  4488,  4196, 16505,  4057,  8066,     3]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [ ]:
outputs = model()

### 입력값 토큰화

In [ ]:
sentences = [
    "Hugging Face는 인공지능(AI)과 자연어 처리(NLP) 분야에서 혁신적인 도구와 모델을 제공하는 AI 스타트업이다.",
    "2016년에 설립된 이 회사는 주로 오픈소스 라이브러리와 사전 학습된 NLP 모델을 제공을 제공한다."
]

### BERT 모델을 이용해 context vector 추출
#### Model 추론결과
- **last_hidden_state**
    - 모든 token들에 대한 feature
    - 출력이 **many**인 작업에 사용한다.
- **pooler_output**
    - 입력 문장, 텍스트에 대한 context vector 이다.
    - 이 값은 **문장을 입력받아 처리하는 task**(ex: 문서분류-감정분석,문장카테고리분류, 문장유사도 분석)의 입력으로 사용한다.

In [29]:
# outputs = model(attention_mask=A, input_ids=I ,token_type_ids=TI)
outputs = model(**token_list)   # token_list: dict -> 가변인자에 item 별로 나눠서 입력.

In [30]:
type(outputs)

transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions

In [31]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [ ]:
outputs['last_hidden_state'].shape
# [3: 문장수-batch, 10:토큰수-> seq-len, 768: 개별토큰의 feature개수]

torch.Size([3, 10, 768])

In [ ]:
outputs['pooler_output'].shape
# 문서(문장)별로 하나씩 -> 문서의 context vector 값 (문서에 대한 feature vector 역할.)

torch.Size([3, 768])

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[    2, 19017,  8482,    17,  1483,  4981,  8046,    17,     3,     0],
        [    2,    76,  4773,  4545, 13146,  4401,  4008,    67, 13146,     3],
        [    2,  3160,  4147, 16991,  4488,  4196, 16505,  4057,  8066,     3]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}